In [0]:
%run "../set-up/mount_data_route"
%run "../utils/analyze_missing_values_fn"

In [0]:
from pyspark.sql.functions import current_timestamp


In [0]:
# Voir les dossier disponible dans le mount
display(dbutils.fs.mounts())

In [0]:
# Chargement des données en tenant compte des bons types de données
credit_df = spark.read.csv(
    f"{bronze_folder_path}/creditcard.csv", 
    header=True, 
    inferSchema=True,
    nullValue="NA",
    nanValue="NaN"
    )

In [0]:
credit_df.printSchema()

In [0]:
# credit_df.show(5, truncate=False)
display(credit_df)

In [0]:
print(f"Nombre de lignes: {credit_df.count()}")
print(f"Nombre de colonnes: {len(credit_df.columns)}")

In [0]:
# credit_df.describe().show()
display(credit_df.describe())

In [0]:
# Analyse des données manquantes
missing_analysis = analyze_missing_values(credit_df)
print(missing_analysis)

In [0]:
# Compter les doublons
print(f"\nNombre de doublons: {credit_df.count() - credit_df.distinct().count()}")

In [0]:
credit_df = credit_df.dropDuplicates()

In [0]:
# Renommer les colonnes
credit_df = credit_df.withColumnRenamed("Class", "Y")

In [0]:
display(credit_df)

In [0]:
credit_df = credit_df.withColumn("indegestion_date", current_timestamp())

In [0]:
credit_df.write.mode('overwrite').parquet(f"{silver_folder_path}/credit_card_data")